In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn

np.random.seed(10)

rn.seed(10)

config = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K
tf.set_random_seed(10)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

C:\Users\kiran\Anaconda3\envs\ML\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kiran\Anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\Users\kiran\Anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [ ]:
filtered_features_data = pd.read_csv("C:/kv/Thesus/Docs/filtered_features_data.csv",header=0)


print(np.unique(filtered_features_data.sub_label,return_counts=True))
print(np.unique(filtered_features_data.main_label,return_counts=True))

In [ ]:
X = filtered_features_data[filtered_features_data.columns.difference(['source_id','sub_label','main_label'])]
#y_main = filtered_features_data.main_label
#y_sub = filtered_features_data.sub_label
y = filtered_features_data[['sub_label','main_label']]


In [ ]:
np.ceil(X.min())

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
scaler = RobustScaler()
#scaler = StandardScaler()
#scaler = MinMaxScaler()
X_scl = scaler.fit_transform(X) #X_train #X_train_res

X_scl = pd.DataFrame(X_scl,columns=X.columns)


In [ ]:
np.ceil(X_scl.min())

In [ ]:
def getBoxPlot(dataset):
    plot=dataset.boxplot(by='sub_label',showmeans=True)
    fig = plot.get_figure()
    fig.suptitle('')

In [ ]:
getBoxPlot(filtered_features_data[['Amplitude','sub_label']])

In [170]:
import numpy as np
import tensorflow as tf
import random as rn

np.random.seed(10)

rn.seed(10)

config = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K
tf.set_random_seed(10)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

In [2]:
import pandas as pd
flat_recall_results = []
flat_precision_results=[]
flat_accuracy_results=[]
flat_balanced_accuracy=[]
flat_f1_val=[]


stack_recall_results = []
stack_precision_results=[]
stack_accuracy_results=[]
stack_balanced_accuracy=[]
stack_f1_val=[]


ancestor_recall_results = []
ancestor_precision_results=[]
ancestor_accuracy_results=[]
ancestor_balanced_accuracy=[]
ancestor_f1_val=[]


parent_recall_results = []
parent_precision_results=[]
parent_accuracy_results=[]
parent_balanced_accuracy=[]
parent_f1_val=[]

target_recall_results = []
target_precision_results=[]
target_accuracy_results=[]
target_balanced_accuracy=[]
target_f1_val=[]

In [3]:
#plastic_data = pd.read_csv('C:/kv/Thesus/PLAStiCC/plasticc_filtered_data.csv',header=0)
plastic_data = pd.read_csv('C:/kv/Thesus/PLAStiCC/plasticc_non_filtered_data.csv',header=0)

In [4]:
#full_pla_data.columns[full_pla_data.isnull().any()]
plastic_data.columns[plastic_data.isnull().any()]
plastic_data = plastic_data.fillna(value=0.0)

In [ ]:
#plastic_data.info()
#plastic_data = plastic_data.fillna(value=0)


#plastic_data = full_pla_data.fillna(value=0.0)

In [5]:
X = plastic_data[plastic_data.columns.difference(['object_id','target','parent_label','ancestor_label'])]
y = plastic_data[['target','parent_label','ancestor_label']]

X=X.astype(float)
print(X.shape)
print(y.shape)
print(np.unique(y.target,return_counts=True))


(4381, 197)
(4381, 3)
(array([42, 52, 62, 67, 90], dtype=int64), array([1193,  183,  484,  208, 2313], dtype=int64))


In [6]:
#Not required now
def flat_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(102,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def stack_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(105,activation='softplus'))

    model.add(Dense(5, activation='softmax'))

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def ancestor_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
   
    model.add(Dense(101,activation='softplus')) #12 10 8 

    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def parent_model():
    model = Sequential()
    model.add(Dense(200, input_dim=199, activation='softplus',kernel_initializer=glorot_normal(seed=100)))

    model.add(Dense(102,activation='softplus')) #12 60 bal

    model.add(Dense(3, activation='softmax'))
    optimizer = Adagrad(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def target_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(105,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model


In [171]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
from keras.layers import Input,Dense,Dropout
from keras.models import Model,Sequential
from keras.initializers import glorot_normal,glorot_uniform
from keras.wrappers.scikit_learn import KerasClassifier
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam,Adagrad
from keras.constraints import maxnorm

def deep_flat_model():
    model = Sequential() #200 150 100 75 25 5
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_stack_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_ancestor_model():
    model = Sequential()
    model.add(Dense(200, input_dim=197, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))


    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_parent_model():
    model = Sequential()
    model.add(Dense(200, input_dim=199, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))


    model.add(Dense(3, activation='softmax'))
    optimizer = Adagrad(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def deep_target_model():
    model = Sequential()
    model.add(Dense(205, input_dim=202, activation='softplus',kernel_initializer=glorot_normal(seed=100)))
    model.add(Dense(175,activation='softplus'))
    model.add(Dense(150,activation='softplus'))
    model.add(Dense(125,activation='softplus'))
    model.add(Dense(100,activation='softplus'))
    model.add(Dense(75,activation='softplus'))
    model.add(Dense(50,activation='softplus'))
    model.add(Dense(25,activation='softplus'))
    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model





def _flat_model():
    model = Sequential()
    model.add(Dense(25, input_dim=20, activation='softplus',kernel_initializer=glorot_normal(seed=10)))
    model.add(Dropout(0.2))
    model.add(Dense(15,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def _ancestor_model():
    model = Sequential()
    model.add(Dense(25, input_dim=20, activation='softplus',kernel_initializer=glorot_normal(seed=10)))
    model.add(Dropout(0.2))
    model.add(Dense(13,activation='softplus')) #12 10 8 

    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def _parent_model():
    model = Sequential()
    model.add(Dense(25, input_dim=22, activation='softplus',kernel_initializer=glorot_normal(seed=10)))
    model.add(Dropout(0.2))
    model.add(Dense(14,activation='softplus')) #12 60 bal

    model.add(Dense(3, activation='softmax'))
    optimizer = Adagrad(lr=0.01)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model

def _target_model():
    model = Sequential()
    model.add(Dense(25, input_dim=25, activation='softplus',kernel_initializer=glorot_normal(seed=10)))
    model.add(Dropout(0.2))
    model.add(Dense(15,activation='softplus'))

    model.add(Dense(5, activation='softmax'))
    optimizer = Adam(lr=0.001)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['sparse_categorical_accuracy'])#sparse_categorical_accuracy
    return model


def getTrainTestData(X,y,random_seed):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=random_seed)
    X_train=X_train.reset_index(drop=True)
    X_test =X_test.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    y_test =y_test.reset_index(drop=True)
    
    return X_train,X_test,y_train,y_test

def performScaling(X_train,X_test,y_train,y_test,col_list):

    from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
    scaler = RobustScaler()
    #scaler = MinMaxScaler()
    X_train_scl = scaler.fit_transform(X_train) #X_train #X_train_res
    X_test_scl = scaler.transform(X_test)

    X_train_scl = pd.DataFrame(X_train_scl, columns = col_list) #X-train_scl
    X_test_scl  = pd.DataFrame(X_test_scl, columns = col_list)

    y_train=y_train.reset_index(drop=True)
    y_test =y_test.reset_index(drop=True)

    scaled_train_df = pd.concat([X_train_scl,y_train],axis=1)
    scaled_test_df = pd.concat([X_test_scl,y_test],axis=1)
    
    return scaled_train_df,scaled_test_df


def get_estimator(model,X,y,weights):
    estimator = train_model(model,X,y)
    estimator.fit(X,y,class_weight=weights)
 
    return estimator

def train_model(base,X,y):
    
    from sklearn.model_selection import StratifiedKFold,cross_val_score  
    estimator = KerasClassifier(build_fn=base, epochs=200, batch_size=100,verbose=0)
   
        
    #kfold = StratifiedKFold(n_splits=10, shuffle=True,random_state=100)#random_state=0
    #results = cross_val_score(estimator,X,y, cv=kfold,scoring='balanced_accuracy')                              
    #print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))  
    print("skipped cv")
    return estimator

def evaluate_model(classifier,X,y):
   
    
    predictions,predict_prob,accuracy,bal_accuracy,precision,recall,f1,confusion_mat = get_metrics(classifier,X,y)
    
    print_metrics(accuracy,bal_accuracy,precision,recall,f1,confusion_mat)
    
    return predictions,predict_prob,accuracy,bal_accuracy,precision,recall,f1,confusion_mat

def get_metrics(classifier,X,y):
    
    from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,balanced_accuracy_score,f1_score
    predictions,predict_prob = get_predictions(classifier,X)
    accuracy = accuracy_score(y,predictions)
    balanced_accuracy= balanced_accuracy_score(y,predictions)
    precision=precision_score(y,predictions,average=None)
    recall = recall_score(y,predictions,average=None)
    confusion_mat = confusion_matrix(y,predictions)
    f1 = f1_score(y, predictions, labels=None, pos_label=1,average=None, sample_weight=None)
    
    return predictions,predict_prob,accuracy,balanced_accuracy,precision,recall,f1,confusion_mat

def get_predictions(classifier,X):
    return classifier.predict(X), classifier.predict_proba(X)


def print_metrics(accuracy,balanced_accuracy,precision,recall,f1,confusion_mat):    
    print(" Accuracy", accuracy)
    print(" Balanced accuracy",balanced_accuracy)
    print(" Recall",recall )
    print(" Precision",precision)
    print("f1",f1)
    print(confusion_mat)
    
    
def getExperimentData(train_df,test_df,label_type):
    
    X_train = train_df[train_df.columns.difference(['ancestor_label','parent_label','target'])]
    X_test = test_df[test_df.columns.difference(['ancestor_label','parent_label','target'])]
    
    if (label_type=="ancestor"):
        
        Y_train = train_df[['ancestor_label']]
        Y_test = test_df[['ancestor_label']]
    
    elif(label_type=="parent"):
       
        Y_train = train_df[['parent_label']]
        Y_test = test_df[['parent_label']]
    
    else:
        Y_train = train_df[['target']]
        Y_test = test_df[['target']]
    
    
    print(label_type + "X_train:", X_train.shape)
    print(label_type + "Y_train:", Y_train.shape)

    print(label_type + "X_test", X_test.shape)
    print(label_type + "Y_test", Y_test.shape)

    print(label_type + "Y_train:", np.unique(Y_train,return_counts=True))
    print(label_type + "Y_test:", np.unique( Y_test,return_counts=True))
    print("\n \n")
    
    return X_train,X_test,Y_train,Y_test
    
    
    
def get_probs(dataset,predict_prob,method):
    if method=="stack":
        ip = pd.DataFrame(predict_prob, columns=['prob1','prob2','prob3','prob4','prob5'])
        X = dataset.copy()
        X['prob1']=ip[['prob1']]
        X['prob2']=ip[['prob2']]
        X['prob3']=ip[['prob3']]
        X['prob4']=ip[['prob4']]
        X['prob5']=ip[['prob5']]
        
    elif method=="ancestor":
        ip = pd.DataFrame(predict_prob, columns=['prob1','prob2'])
        X = dataset.copy()
        X['prob1']=ip[['prob1']]
        X['prob2']=ip[['prob2']]
    
    elif method=="parent":
        ip = pd.DataFrame(predict_prob, columns=['prob3','prob4','prob5'])
        X = dataset.copy()
        X['prob3']=ip[['prob3']]
        X['prob4']=ip[['prob4']]
        X['prob5']=ip[['prob5']]

        
    return X

def put_in_df(metric,hierarchy):
    
    if (hierarchy=="accuracy"):
        headers = ['accuracy','bal_accuracy']
    elif (hierarchy=="ancestor"):
        headers = ['class42','class100']
    elif (hierarchy=="parent"):
        
        headers = ['class42','class62','class90']
        
    else:
        headers= ['class42','class52','class62','class67','class90']
    
    df = pd.DataFrame(metric,columns=headers)
    
    return df


In [119]:
#del parent_X_train,parent_X_test,parent_Y_train,parent_Y_test
#del target_X_train,target_X_test,target_Y_train,target_Y_test

In [172]:
col = X.columns
X_train,X_test,y_train,y_test = getTrainTestData(X,y,100)

scaled_train_df, scaled_test_df = performScaling(X_train,X_test,y_train,y_test,col)

flat_X_train,flat_X_test,flat_Y_train,flat_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"flat")
ancestor_X_train,ancestor_X_test,ancestor_Y_train,ancestor_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"ancestor")
#parent_X_train,parent_X_test,parent_Y_train,parent_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"parent")
#target_X_train,target_X_test,target_Y_train,target_Y_test = getExperimentData(scaled_train_df, scaled_test_df,"target")

#del scaled_train_df, scaled_test_df

flatX_train: (3285, 197)
flatY_train: (3285, 1)
flatX_test (1096, 197)
flatY_test (1096, 1)
flatY_train: (array([42, 52, 62, 67, 90], dtype=int64), array([ 898,  141,  360,  140, 1746], dtype=int64))
flatY_test: (array([42, 52, 62, 67, 90], dtype=int64), array([295,  42, 124,  68, 567], dtype=int64))

 

ancestorX_train: (3285, 197)
ancestorY_train: (3285, 1)
ancestorX_test (1096, 197)
ancestorY_test (1096, 1)
ancestorY_train: (array([ 42, 100], dtype=int64), array([ 898, 2387], dtype=int64))
ancestorY_test: (array([ 42, 100], dtype=int64), array([295, 801], dtype=int64))

 



In [173]:
weights={}

flat_estimator = get_estimator(deep_flat_model,flat_X_train,flat_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

flat_train_predictions,flat_train_predict_prob = get_predictions(flat_estimator,flat_X_train)

flat_predictions,flat_predict_prob,flat_accuracy,flat_bal_accuracy,flat_precision,flat_recall,flat_f1,flat_confusion_mat=evaluate_model(flat_estimator,flat_X_test,flat_Y_test)

flat_recall_results.append(flat_recall)
flat_precision_results.append(flat_precision)
flat_accuracy_results.append([flat_accuracy,flat_bal_accuracy])
flat_f1_val.append(flat_f1)




skipped cv
 Accuracy 0.7244525547445255
 Balanced accuracy 0.5890094579435161
 Recall [0.61016949 0.30952381 0.55645161 0.60294118 0.8659612 ]
 Precision [0.6870229  0.40625    0.60526316 0.68333333 0.78184713]
f1 [0.64631957 0.35135135 0.57983193 0.640625   0.82175732]
[[180   3  23   5  84]
 [  8  13   9   0  12]
 [ 17   2  69   7  29]
 [  6   2   7  41  12]
 [ 51  12   6   7 491]]


In [174]:
flat_accuracy_results

[[0.7235401459854015, 0.5362469933400869],
 [0.7271897810218978, 0.5864335837643185],
 [0.7144160583941606, 0.5508466026080988],
 [0.7299270072992701, 0.563215474991772],
 [0.7208029197080292, 0.5354023004226104],
 [0.7052919708029197, 0.5331797186179359],
 [0.708941605839416, 0.5343011805366876],
 [0.6806569343065694, 0.5510141996527025],
 [0.7235401459854015, 0.5631465600142679],
 [0.7244525547445255, 0.5890094579435161]]

In [175]:
stack_X_train = get_probs(flat_X_train,flat_train_predict_prob,"stack")
stack_X_test = get_probs(flat_X_test,flat_predict_prob,"stack")


stack_Y_train= scaled_train_df[['target']]
stack_Y_test = scaled_test_df[['target']]


print(stack_X_train.shape)
print(stack_X_test.shape)
print(np.unique(stack_Y_train,return_counts=True))

print(np.unique(stack_Y_test, return_counts=True))

(3285, 202)
(1096, 202)
(array([42, 52, 62, 67, 90], dtype=int64), array([ 898,  141,  360,  140, 1746], dtype=int64))
(array([42, 52, 62, 67, 90], dtype=int64), array([295,  42, 124,  68, 567], dtype=int64))


In [176]:
weights={}

stack_estimator = get_estimator(deep_stack_model,stack_X_train,stack_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

stack_train_predictions,stack_train_predict_prob = get_predictions(stack_estimator,stack_X_train)

stack_predictions,stack_predict_prob,stack_accuracy,stack_bal_accuracy,stack_precision,stack_recall,stack_f1,stack_confusion_mat=evaluate_model(stack_estimator,stack_X_test,stack_Y_test)

stack_recall_results.append(stack_recall)
stack_precision_results.append(stack_precision)
stack_accuracy_results.append([stack_accuracy,stack_bal_accuracy])
stack_f1_val.append(stack_f1)

skipped cv
 Accuracy 0.7262773722627737
 Balanced accuracy 0.5969242151227379
 Recall [0.72542373 0.30952381 0.5483871  0.58823529 0.81305115]
 Precision [0.60623229 0.36111111 0.60714286 0.70175439 0.85687732]
f1 [0.66049383 0.33333333 0.57627119 0.64       0.83438914]
[[214   4  23   5  49]
 [ 10  13   8   0  11]
 [ 36   3  68   7  10]
 [ 10   4   7  40   7]
 [ 83  12   6   5 461]]


In [177]:
#stack_accuracy_results.pop()
#stack_precision_results.pop()
#stack_recall_results.pop()
#stack_f1_val.pop()
stack_accuracy_results

[[0.7235401459854015, 0.5294421094231463],
 [0.7226277372262774, 0.5808526173231643],
 [0.7052919708029197, 0.5339351407000686],
 [0.7226277372262774, 0.5553448695853362],
 [0.7153284671532847, 0.541241923414332],
 [0.698905109489051, 0.5462321930206662],
 [0.7107664233576643, 0.5423266230145638],
 [0.698905109489051, 0.5263465613098772],
 [0.7262773722627737, 0.5463836210554318],
 [0.7262773722627737, 0.5969242151227379]]

In [178]:
weights={}

ancestor_estimator = get_estimator(deep_ancestor_model,ancestor_X_train,ancestor_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

ancestor_train_predictions,ancestor_train_predict_prob = get_predictions(ancestor_estimator,ancestor_X_train)

ancestor_predictions,ancestor_predict_prob,ancestor_accuracy,ancestor_bal_accuracy,ancestor_precision,ancestor_recall,ancestor_f1,ancestor_confusion_mat=evaluate_model(ancestor_estimator,ancestor_X_test,ancestor_Y_test)

ancestor_recall_results.append(ancestor_recall)
ancestor_precision_results.append(ancestor_precision)
ancestor_accuracy_results.append([ancestor_accuracy,ancestor_bal_accuracy])
ancestor_f1_val.append(ancestor_f1)

skipped cv
 Accuracy 0.8166058394160584
 Balanced accuracy 0.7406948940942466
 Recall [0.57627119 0.9051186 ]
 Precision [0.69105691 0.85294118]
f1 [0.6284658 0.8782556]
[[170 125]
 [ 76 725]]


In [179]:
ancestor_accuracy_results

[[0.802007299270073, 0.7810860083145914],
 [0.823905109489051, 0.7385575177702004],
 [0.7956204379562044, 0.7275958302552737],
 [0.7928832116788321, 0.7305850123718629],
 [0.8047445255474452, 0.7336838288614937],
 [0.8138686131386861, 0.6995651970639114],
 [0.7974452554744526, 0.7155605670103093],
 [0.7846715328467153, 0.750088193166411],
 [0.7947080291970803, 0.6981398544455188],
 [0.8166058394160584, 0.7406948940942466]]

In [180]:
parent_X_train = get_probs(ancestor_X_train,ancestor_train_predict_prob,"ancestor")
parent_X_test = get_probs(ancestor_X_test,ancestor_predict_prob,"ancestor")


parent_Y_train= scaled_train_df[['parent_label']]
parent_Y_test = scaled_test_df[['parent_label']]


print(parent_X_train.shape)
print(parent_X_test.shape)
print(np.unique(parent_Y_train,return_counts=True))

print(np.unique(parent_Y_test, return_counts=True))

(3285, 199)
(1096, 199)
(array([42, 62, 90], dtype=int64), array([ 898,  360, 2027], dtype=int64))
(array([42, 62, 90], dtype=int64), array([295, 124, 677], dtype=int64))


In [181]:
weights={}

parent_estimator = get_estimator(deep_parent_model,parent_X_train,parent_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

parent_train_predictions,parent_train_predict_prob = get_predictions(parent_estimator,parent_X_train)

parent_predictions,parent_predict_prob,parent_accuracy,parent_bal_accuracy,parent_precision,parent_recall,parent_f1,parent_confusion_mat=evaluate_model(parent_estimator,parent_X_test,parent_Y_test)

parent_recall_results.append(parent_recall)
parent_precision_results.append(parent_precision)
parent_accuracy_results.append([parent_accuracy,parent_bal_accuracy])
parent_f1_val.append(parent_f1)

skipped cv
 Accuracy 0.7518248175182481
 Balanced accuracy 0.6355774586312377
 Recall [0.60338983 0.42741935 0.87592319]
 Precision [0.70355731 0.50961538 0.80243572]
f1 [0.64963504 0.46491228 0.83757062]
[[178  21  96]
 [ 21  53  50]
 [ 54  30 593]]


In [182]:
parent_accuracy_results

[[0.7655109489051095, 0.6655099159709276],
 [0.7408759124087592, 0.6591015239202872],
 [0.7317518248175182, 0.6414546990221875],
 [0.7290145985401459, 0.6058000658333148],
 [0.7536496350364964, 0.6497131429846722],
 [0.7427007299270073, 0.6114349218387899],
 [0.7618613138686131, 0.6423009500960154],
 [0.7390510948905109, 0.6472206256831674],
 [0.7518248175182481, 0.6355774586312377]]

In [183]:
target_X_train = get_probs(parent_X_train,parent_train_predict_prob,"parent")
target_X_test = get_probs(parent_X_test,parent_predict_prob,"parent")

target_Y_train= scaled_train_df[['target']]
target_Y_test = scaled_test_df[['target']]

print(target_X_train.shape)
print(target_X_test.shape)
print(np.unique(target_Y_train,return_counts=True))
print(np.unique(target_Y_test, return_counts=True))

(3285, 202)
(1096, 202)
(array([42, 52, 62, 67, 90], dtype=int64), array([ 898,  141,  360,  140, 1746], dtype=int64))
(array([42, 52, 62, 67, 90], dtype=int64), array([295,  42, 124,  68, 567], dtype=int64))


In [184]:
weights={}

target_estimator = get_estimator(deep_target_model,target_X_train,target_Y_train,weights)
#base_estimator = get_estimator(base_class_model,base_X_train,base_Y_train,50,weights)

target_train_predictions,target_train_predict_prob = get_predictions(target_estimator,target_X_train)

target_predictions,target_predict_prob,target_accuracy,target_bal_accuracy,target_precision,target_recall,target_f1,target_confusion_mat=evaluate_model(target_estimator,target_X_test,target_Y_test)

target_recall_results.append(target_recall)
target_precision_results.append(target_precision)
target_accuracy_results.append([target_accuracy,target_bal_accuracy])
target_f1_val.append(target_f1)

skipped cv
 Accuracy 0.708029197080292
 Balanced accuracy 0.5678855684184858
 Recall [0.69830508 0.35714286 0.48387097 0.48529412 0.81481481]
 Precision [0.59195402 0.45454545 0.55045872 0.55       0.84615385]
f1 [0.6407465  0.4        0.51502146 0.515625   0.83018868]
[[206   5  23   6  55]
 [  9  15   4   3  11]
 [ 40   4  60  10  10]
 [ 11   1  15  33   8]
 [ 82   8   7   8 462]]


In [185]:
flat_accuracy_results


[[0.7235401459854015, 0.5362469933400869],
 [0.7271897810218978, 0.5864335837643185],
 [0.7144160583941606, 0.5508466026080988],
 [0.7299270072992701, 0.563215474991772],
 [0.7208029197080292, 0.5354023004226104],
 [0.7052919708029197, 0.5331797186179359],
 [0.708941605839416, 0.5343011805366876],
 [0.6806569343065694, 0.5510141996527025],
 [0.7235401459854015, 0.5631465600142679],
 [0.7244525547445255, 0.5890094579435161]]

In [186]:
stack_accuracy_results

[[0.7235401459854015, 0.5294421094231463],
 [0.7226277372262774, 0.5808526173231643],
 [0.7052919708029197, 0.5339351407000686],
 [0.7226277372262774, 0.5553448695853362],
 [0.7153284671532847, 0.541241923414332],
 [0.698905109489051, 0.5462321930206662],
 [0.7107664233576643, 0.5423266230145638],
 [0.698905109489051, 0.5263465613098772],
 [0.7262773722627737, 0.5463836210554318],
 [0.7262773722627737, 0.5969242151227379]]

In [187]:
target_accuracy_results

[[0.7135036496350365, 0.5107992320430241],
 [0.7135036496350365, 0.5548645358215778],
 [0.6916058394160584, 0.5209224433768016],
 [0.6852189781021898, 0.5217055330738709],
 [0.6961678832116789, 0.5307008662567562],
 [0.6833941605839416, 0.5671132406539149],
 [0.697992700729927, 0.5289328908416683],
 [0.6934306569343066, 0.5270292741597402],
 [0.6888686131386861, 0.5241188659259465],
 [0.708029197080292, 0.5678855684184858]]

In [ ]:
flat_precision_results

In [ ]:
stack_precision_results

In [ ]:
target_precision_results

In [ ]:
flat_recall_results

In [ ]:
stack_recall_results

In [ ]:
target_recall_results

In [ ]:
flat_f1_val

In [ ]:
stack_f1_val

In [ ]:
target_f1_val

In [188]:

ancestor_precision_df=put_in_df(ancestor_precision_results,"ancestor")
ancestor_recall_df=put_in_df(ancestor_recall_results,"ancestor")
ancestor_f1_df=put_in_df(ancestor_f1_val,"ancestor")


parent_precision_df=put_in_df(parent_precision_results,"parent")
parent_recall_df=put_in_df(parent_recall_results,"parent")
parent_f1_df=put_in_df(parent_f1_val,"parent")

target_precision_df=put_in_df(target_precision_results,"target")
target_recall_df=put_in_df(target_recall_results,"target")
target_f1_df=put_in_df(target_f1_val,"target")


flat_precision_df=put_in_df(flat_precision_results,"flat")
flat_recall_df=put_in_df(flat_recall_results,"flat")
flat_f1_df=put_in_df(flat_f1_val,"flat")


stack_precision_df=put_in_df(stack_precision_results,"flat")
stack_recall_df=put_in_df(stack_recall_results,"flat")
stack_f1_df=put_in_df(stack_f1_val,"flat")


flat_acc_df = put_in_df(flat_accuracy_results,"accuracy")
stack_acc_df = put_in_df(stack_accuracy_results,"accuracy")
ancestor_acc_df = put_in_df(ancestor_accuracy_results,"accuracy")
parent_acc_df = put_in_df(parent_accuracy_results,"accuracy")
target_acc_df = put_in_df(target_accuracy_results,"accuracy")

flat_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_flat_precision.csv')
flat_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_flat_recall.csv')
flat_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_flat_f1.csv')

stack_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_stack_precision.csv')
stack_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_stack_recall.csv')
stack_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_stack_f1.csv')


ancestor_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_ancestor_precision.csv')
ancestor_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_ancestor_recall.csv')
ancestor_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_ancestor_f1.csv')


parent_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_parent_precision.csv')
parent_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_parent_recall.csv')
parent_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_parent_f1.csv')


target_precision_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_target_precision.csv')
target_recall_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_target_recall.csv')
target_f1_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_target_f1.csv')

flat_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_flat_acc.csv')
stack_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_stack_acc.csv')
ancestor_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_ancestor_acc.csv')
parent_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_parent_acc.csv')
target_acc_df.to_csv('C:/kv/Thesus/PLAStiCC/results/deep_target_acc.csv')


del stack_precision_df,stack_recall_df,stack_f1_df,flat_precision_df,flat_recall_df,flat_f1_df,ancestor_precision_df,ancestor_recall_df,ancestor_f1_df,parent_precision_df,parent_recall_df,parent_f1_df,
target_precision_df,target_recall_df,target_f1_df,flat_acc_df,stack_acc_df,ancestor_acc_df,parent_acc_df,parent_acc_df

(    class42   class52   class62   class67   class90
 0  0.709163  0.243243  0.626263  0.477273  0.769925
 1  0.695312  0.236842  0.627119  0.507692  0.788368
 2  0.652482  0.258065  0.516667  0.470588  0.784314
 3  0.543536  0.428571  0.467213  0.547619  0.857143
 4  0.608069  0.407407  0.521368  0.434783  0.822898
 5  0.643939  0.400000  0.486726  0.370787  0.805085
 6  0.625714  0.266667  0.495413  0.434783  0.842491
 7  0.573446  0.250000  0.485981  0.641026  0.836879
 8  0.566138  0.208333  0.504505  0.682927  0.833948
 9  0.591954  0.454545  0.550459  0.550000  0.846154,
     class42   class52   class62   class67   class90
 0  0.609589  0.191489  0.484375  0.375000  0.893543
 1  0.605442  0.191489  0.560606  0.550000  0.866785
 2  0.593548  0.160000  0.500000  0.500000  0.851064
 3  0.746377  0.187500  0.471074  0.442308  0.761269
 4  0.685065  0.250000  0.530435  0.392157  0.795848
 5  0.580205  0.372093  0.429688  0.634615  0.818966
 6  0.684375  0.307692  0.446281  0.392157  0

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score,make_scorer,f1_score

model = KerasClassifier(build_fn=flat_model,verbose=0)
batch_size = [100, 500, 1000, 2000]
epochs = [100,200,300]
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#param_grid = dict(optimizer=optimizer)
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
#units = [8,10,12,14,16]
#param_grid = dict(units=units)
#weight_constraint = [1, 2, 3, 4, 5]
#dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(batch_size=batch_size,epochs=epochs)
#scoring = {'bal_accuracy': make_scorer(balanced_accuracy_score)}
#f1_scorer = make_scorer(f1_score,average='micro',greater_is_better=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(flat_X_train,flat_Y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:

#Best: 0.701065 using {'optimizer': 'Adam'}
#0.666058 (0.011317) with: {'optimizer': 'SGD'}
#0.688280 (0.012305) with: {'optimizer': 'RMSprop'}
#0.674886 (0.008202) with: {'optimizer': 'Adagrad'}
#0.692542 (0.007165) with: {'optimizer': 'Adadelta'}
#0.701065 (0.007761) with: {'optimizer': 'Adam'}
#0.686454 (0.008642) with: {'optimizer': 'Adamax'}
#0.687671 (0.008066) with: {'optimizer': 'Nadam'}


#Best: 0.690107 using {'learn_rate': 0.001}
#0.690107 (0.006766) with: {'learn_rate': 0.001}
#0.656621 (0.006501) with: {'learn_rate': 0.01}
#0.603957 (0.028003) with: {'learn_rate': 0.1}
#0.542770 (0.012705) with: {'learn_rate': 0.2}
#0.512329 (0.032133) with: {'learn_rate': 0.3}



#Best: 0.704110 using {'init_mode': 'normal'}
#0.693455 (0.010607) with: {'init_mode': 'uniform'}
#0.687976 (0.006027) with: {'init_mode': 'lecun_uniform'}
#0.704110 (0.016856) with: {'init_mode': 'normal'}
#0.529680 (0.009041) with: {'init_mode': 'zero'}
#0.697717 (0.008469) with: {'init_mode': 'glorot_normal'}
#0.695586 (0.008214) with: {'init_mode': 'glorot_uniform'}
#0.693151 (0.012455) with: {'init_mode': 'he_normal'}
#0.692542 (0.006073) with: {'init_mode': 'he_uniform'}

#Best: 0.702892 using {'activation': 'softplus'}
#0.654795 (0.009163) with: {'activation': 'softmax'}
#0.702892 (0.003444) with: {'activation': 'softplus'}
#0.698630 (0.006834) with: {'activation': 'softsign'}
#0.689193 (0.018300) with: {'activation': 'relu'}
#0.695282 (0.008642) with: {'activation': 'tanh'}
#0.691020 (0.004965) with: {'activation': 'sigmoid'}
#0.683714 (0.005496) with: {'activation': 'hard_sigmoid'}
#0.650228 (0.001292) with: {'activation': 'linear'}


#Best: 0.708371 using {'units': 12}
#0.699239 (0.009442) with: {'units': 8}
#0.698630 (0.002237) with: {'units': 10}
#0.708371 (0.007616) with: {'units': 12}
#0.706240 (0.006766) with: {'units': 14}
#0.705632 (0.005792) with: {'units': 16}